In [1]:
# since we are in a subfolder
from utils import append_sys_path

append_sys_path()

INFO:root:Appended /home/laci/repos/agent-guy to sys.path


In [ ]:
import random
from agent_guy.agent import IAgent, ITurtle, IPatch
from agent_guy.world import Grid

from agent_guy.executor import SequentialExecutor, RandomExecutor
from agent_guy.model import IModel
from agent_guy.runner import Runner
from agent_guy.visitor import CsvVisitor
from agent_guy.observer import IObserver

In [ ]:
class LifeCell(ITurtle):
    def __init__(self, x: int, y: int) -> None:
        # a custom variable from the user
        self.neighbors_alive = 0
        self.status = "dead"
        super().__init__(x, y)

    # these are custom functions for the user
    def observe(self) -> None:
        # retruns me the neighbors agent objects
        # there could be also neighbors patches
        neighbors = self.neighbor_turtles()
        # count the number of alive neighbors
        self.neighbors_alive = len([n for n in neighbors if n.status == "alive"])

        return

    def act(self) -> None:
        # if I have less than 2 neighbors I die
        if self.neighbors_alive < 2:
            self.die()
        # if I have more than 3 neighbors I die
        elif self.neighbors_alive > 3:
            self.die()
        # if I have 3 neighbors I live
        elif self.neighbors_alive == 3:
            self.live()
        # otherwise I do nothing
        else:
            pass
        return

    def die(self) -> None:
        self.status = "dead"
        self.set_color("red")
        return

    def live(self) -> None:
        self.status = "alive"
        self.set_color("green")
        return


class LifeModel(IModel):
    def __init__(self, world: IWorld, parameters: dict = dict()):
        super().__init__(world, parameters)

    def setup_world(self) -> None:
        pass  # nothing to do here

    def setup_agents(self) -> None:
        # put an agent on each patch
        for patch in self.world.patches:
            cell = LifeCell(self.world, patch.x, patch.y)

            # pull out of a distribution if the cell is alive or dead
            if random.random() < parameters["chance_of_life"]:
                cell.live()
            else:
                cell.die()

            # add the cell to the world
            self.world.add_turtle(cell)

        return

    def step(self) -> None:
        # all agents observe
        self.world.ask_turtles(funcs=["observe"])

        # all agents act
        self.world.ask_turtles(funcs=["act"])

        return


class Observer(IObserver):
    def aggregate(self, model: IModel) -> dict:
        # get number of turtles alive
        turtles_count = model.world.count_turtles()
        # get number of alive turtles
        turtles_alive = len(model.world.get_turtles(dict(status="alive")))
        # get number of dead turtles
        turtles_dead = turtles_count - turtles_alive

        return {"turtles_alive": turtles_alive, "turtles_dead": turtles_dead}

In [ ]:
runner = Runner(
    model=Model(
        world=Grid(10, 10),
        parameters={
            chance_of_life=0.1,
        }
    ),
    executor=SequentialExecutor(max_step = 100),
    observer=Observer(),
    visitor=CsvVisitor(),
)